In [34]:
# !pip install lark-parser

In [35]:
import subprocess

def run_ltl2ldba(path_to_rabinizer,formula):
    """
    Run the ltl2ldba command with the given LTL formula.

    :param path_to_owl: path to the executable, e.g., ./owl
    :param formula: The LTL formula as a string, e.g.,
    :return: The output from the Owl command, if an error occurs, the error message is returned, which starts with 'Error: '.
    """
    command = [path_to_rabinizer, "ltl2ldgba", "-f", formula]
    
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        if result.returncode == 0:
            return result.stdout
        else:
            return f"Error: {result.stderr}"
    except FileNotFoundError:
        return "Error: Owl binary not found or not executable."
    except Exception as e:
        return f"Error: {e}"

In [36]:
from lark import Lark, Transformer, Tree, Token


hoa_grammar = r"""
    start: header "--BODY--" body "--END--"
    
    header: version tool? name? owl_args? start_state acc_name acceptance properties? ap_decl

    version: "HOA:" /v\d+/
    tool: "tool:" ESCAPED_STRING+
    name: "name:" ESCAPED_STRING
    owl_args: "owlArgs:" ESCAPED_STRING+
    start_state: "Start:" INT
    acc_name: "acc-name:" ("generalized-Buchi" INT | "Buchi")
    acceptance: "Acceptance:" INT acceptance_cond*
    properties: ("Properties:" LOWER_STRING+)+
    ap_decl: "AP:" INT (ESCAPED_STRING)*

    acceptance_cond: "Inf(" INT ")" ("&" "Inf(" INT ")")*

    body: state*
    state: "State:" INT transition* 
    transition: "[" label_expr "]" INT acc_sig?

    label_expr: expr
    expr: factor (LOGIC_OP factor)*
    factor: LOGIC_NOT factor
          | L_PAR expr R_PAR
          | INT
          | IDENTIFIER
    acc_sig: "{" INT+ "}"
    
    IDENTIFIER: /[a-zA-Z_][0-9a-zA-Z_-]*/
    STRING: /[a-zA-Z_-]+/
    LOWER_STRING: /[0-9a-z_-]+/
    LOGIC_OP: "&" | "|"
    LOGIC_NOT: "!"
    L_PAR: "("
    R_PAR: ")"
    
    %import common.ESCAPED_STRING
    %import common.INT
    %import common.WS
    %ignore WS
"""

class HOA_Transformer(Transformer):
    def start(self, items):
        return {
            'header': items[0],
            'body': items[1]
        }
    
    def header(self, items):
        return {
            'version': items[0],
            'tool': items[1] if len(items) > 1 else None,
            'name': items[2] if len(items) > 2 else None,
            'owl_args': items[3] if len(items) > 3 else None,
            'start_state': items[4],
            'acc_name': items[5],
            'acceptance': items[6],
            'properties': items[7],
            'ap_decl': items[8]
        }
    
    def body(self, items):
        return {'states': items}
    
    def state(self, items):
        return {
            'state_id': items[0],
            'transitions': items[1:]
        }

    def transition(self, items):
        return {
            'label': items[0],
            'destination': items[1],
            'acc_sig': items[2] if len(items) > 2 else None
        }

    def acceptance_cond(self, items):
        return [f"{i}" for i in items]

    def ap_decl(self, items):
        count = items[0]
        propositions = {items[i].replace('"', ''): i - 1 for i in range(1, len(items))}
        return {'count': count, 'propositions': propositions}

hoa_parser = Lark(hoa_grammar, parser='lalr', transformer=HOA_Transformer())


hoa_input = '''
HOA: v1
tool: "owl ltl2ldgba" "21.0"
name: "Automaton for ((G(a)) & (G(((b) U (((c) U (((d) | (e)))))))))"
owlArgs: "ltl2ldgba" "-f" "G (a & (b U (c U (d | e))))" 
Start: 0
acc-name: generalized-Buchi 2
Acceptance: 2 Inf(0) & Inf(1)
properties: trans-acc no-univ-branch 
AP: 5 "a" "b" "c" "d" "e"
--BODY--
State: 0
[0 & !1 & 2 & !3 & !4] 1
[0 & (1 | !1 & (3 | !3 & 4))] 0
[0 & !1 & 2 & !3 & !4] 2
[0 & !1 & 2 & !3 & !4] 3
[0 & (1 | !1 & (3 | !3 & 4))] 4
[0 & (1 | !1 & (3 | !3 & 4))] 5
State: 1
[0 & !3 & 4] 6
[0 & 2 & !3 & !4] 1
[0 & 2 & !3 & !4] 7
[0 & (3 | !3 & 4)] 0
[0 & 3 & !4] 8
[0 & 2 & !3 & !4] 9
[0 & 3] 4
[0 & 4] 5
State: 2
[0 & 2 & !3 & !4] 2
[0 & 4] 5 {0 1}
[0 & 3 & !4] 5 {0}
State: 3
[0 & !3 & 4] 4 {0}
[0 & 2 & !3 & !4] 3
[0 & 3] 4 {0 1}
State: 4
[0 & (1 & !3 | !1 & !3 & 4)] 4 {0}
[0 & 3] 4 {0 1}
[0 & !1 & 2 & !3 & !4] 3
State: 5
[0 & !1 & 2 & !3 & !4] 2
[0 & 4] 5 {0 1}
[0 & (1 & !4 | !1 & 3 & !4)] 5 {0}
State: 6
[0 & (1 & !3 | !1 & !3 & 4)] 6
[0 & !1 & 2 & !3 & !4] 10
[0 & 3] 4
State: 7
[0 & !3 & 4] 6
[0 & 2 & !3 & !4] 7
[0 & 3] 4
State: 8
[0 & !1 & 2 & !3 & !4] 11
[0 & (1 & !4 | !1 & 3 & !4)] 8
[0 & 4] 5
State: 9
[0 & 3 & !4] 8
[0 & 2 & !3 & !4] 9
[0 & 4] 5
State: 10
[0 & !3 & 4] 6
[0 & 2 & !3 & !4] 10
[0 & 3] 4
State: 11
[0 & 2 & !3 & !4] 11
[0 & 3 & !4] 8
[0 & 4] 5
--END--
'''

parsed_hoa = hoa_parser.parse(hoa_input.replace("properties", "Properties"))
print(parsed_hoa)

{'header': {'version': Tree('version', [Token('__ANON_3', 'v1')]), 'tool': Tree('tool', [Token('ESCAPED_STRING', '"owl ltl2ldgba"'), Token('ESCAPED_STRING', '"21.0"')]), 'name': Tree('name', [Token('ESCAPED_STRING', '"Automaton for ((G(a)) & (G(((b) U (((c) U (((d) | (e)))))))))"')]), 'owl_args': Tree('owl_args', [Token('ESCAPED_STRING', '"ltl2ldgba"'), Token('ESCAPED_STRING', '"-f"'), Token('ESCAPED_STRING', '"G (a & (b U (c U (d | e))))"')]), 'start_state': Tree('start_state', [Token('INT', '0')]), 'acc_name': Tree('acc_name', [Token('INT', '2')]), 'acceptance': Tree('acceptance', [Token('INT', '2'), ['0', '1']]), 'properties': Tree('properties', [Token('LOWER_STRING', 'trans-acc'), Token('LOWER_STRING', 'no-univ-branch')]), 'ap_decl': {'count': Token('INT', '5'), 'propositions': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}}}, 'body': {'states': [{'state_id': Token('INT', '0'), 'transitions': [{'label': Tree('label_expr', [Tree('expr', [Tree('factor', [Token('INT', '0')]), Token('LOGIC_O

In [37]:
from dataclasses import dataclass

@dataclass
class HOAAutomataTransition:
    label: str
    destination: str
    accepting_signature: list[str]
    
    def __post_init__(self):
        if self.label in ["t"]:
            self.label = "any"
    
    def __str__(self):
        return f"{self.label} -> ({self.destination})" + (f" ACC[{', '.join(self.accepting_signature)}]" if self.accepting_signature else "")
    

@dataclass
class HOAAutomataState:
    state_id: str
    transitions: list[HOAAutomataTransition]
    
    def __str__(self):
        return f"{'('+self.state_id+')':<6}" + ("\n"+6*" ").join([f"-> {tr}" for tr in self.transitions])
    

In [38]:
class HOAParsedHeaderHelper:
    
    @staticmethod
    def extract_start_state_id(parsed_tree):
        return str(parsed_tree['header']['start_state'].children[0].value)

    @staticmethod
    def extract_accepting_sink_sets_id(parsed_tree):
        return [
            str(ch)
            for ch in parsed_tree['header']['acceptance'].children[1:][0]
        ]
    
    @staticmethod
    def extract_atomic_propositions_to_symbol(parsed_tree):
        return {
            str(k): int(v)
            for k, v in parsed_tree['header']['ap_decl']['propositions'].items()
        }
    
    @staticmethod
    def extract_useful_header_info(parsed_tree):
        return {
            'start_state_id': HOAParsedHeaderHelper.extract_start_state_id(parsed_tree),
            'accepting_sink_sets_id': HOAParsedHeaderHelper.extract_accepting_sink_sets_id(parsed_tree),
            'atomic_symbol_to_propositions': HOAParsedHeaderHelper.extract_atomic_propositions_to_symbol(parsed_tree)
        }


HOAParsedHeaderHelper.extract_useful_header_info(parsed_hoa)

{'start_state_id': '0',
 'accepting_sink_sets_id': ['0', '1'],
 'atomic_symbol_to_propositions': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}}

In [39]:
parsed_hoa['header']['acceptance'].children[1:][0]

['0', '1']

In [40]:
class HOAParsedBodyHelper:
    
    @staticmethod
    def _extract_state_id(parsed_state):
        return parsed_state['state_id'].value
    
    @staticmethod
    def _label_walk_helper(transition_label):
        """
        Recursively walks through the parsed tree of a transition label
        and converts it into a string representation.
        """
        if isinstance(transition_label, Tree):
            return "".join([HOAParsedBodyHelper._label_walk_helper(child) for child in transition_label.children])
        elif isinstance(transition_label, Token):
            return transition_label.value
        return str(transition_label)
    
    @staticmethod
    def _extract_acc_sig(acc_sig):
        if acc_sig is None:
            return []
        return [
            ch.value for ch in acc_sig.children
        ]
    
    @staticmethod
    def extract_transitions(state_transitions):
        return [
            HOAAutomataTransition(
                label=HOAParsedBodyHelper._label_walk_helper(tr['label']),
                destination=tr['destination'],
                accepting_signature=HOAParsedBodyHelper._extract_acc_sig(tr['acc_sig'])
            )
            for tr in state_transitions
        ]
    
    @staticmethod
    def extract_states(parsed_tree):
        return [
            HOAAutomataState(
                state_id=HOAParsedBodyHelper._extract_state_id(st),
                transitions=HOAParsedBodyHelper.extract_transitions(st['transitions'])
            )
            for st in parsed_tree['body']['states']
        ]
    
# for st in HOAParsedBodyHelper.extract_states(parsed_hoa):
#     print(st)

In [41]:
@dataclass
class HOAParserHelper:
    parser: Lark
    
    def __call__(self, hoa_format_ldba):
        ldba = self.parser.parse(hoa_format_ldba.replace("properties", "Properties"))
        return {
            'header': HOAParsedHeaderHelper.extract_useful_header_info(ldba),
            'states': HOAParsedBodyHelper.extract_states(ldba)
        }
        

In [42]:
from enum import Enum
import json

class AutomataTransitionType(Enum):
    Any = "any"
    Epsilon = "epsilon"
    Propositional = "propositional"
    
    @classmethod
    def from_str(cls, value: str):
        if value in ["any", "t"]:
            return cls.Any
        if value in ["epsilon", "e"] or len(value) == 0:
            return cls.Epsilon
        return cls.Propositional
    
class AutomataStateStat(Enum):
    Accepting = "accepting"
    NonAccepting = "non-accepting"
    
    def __str__(self):
        if self == AutomataStateStat.Accepting:
            return "(●)"
        return "( )"
    
    @classmethod
    def from_str(cls, value: str):
        if value in ["accepting"]:
            return cls.Accepting
        return cls.NonAccepting

@dataclass
class AutomataTransition:
    type: AutomataTransitionType
    destination_id: str
    predicate: str
    
    def __post_init__(self):
        self.predicate = self.predicate.replace("&", " & ").replace("|", " | ")
        
    def __str__(self):
        return f"--[{self.predicate:^5}]--> {self.destination_id}"
    

@dataclass
class AutomataState:
    state_id: str
    status: AutomataStateStat
    transitions: list[AutomataTransition]
    accepting_signature: list[str]
    
    def __post_init__(self):
        if self.status == AutomataStateStat.Accepting and len(self.accepting_signature) == 0:
            raise ValueError("Accepting state must have an accepting signature.")
        if self.status == AutomataStateStat.NonAccepting and len(self.accepting_signature) > 0:
            raise ValueError("Non-accepting state cannot have an accepting signature.")
    
    def is_accepting(self):
        return self.status == AutomataStateStat.Accepting
    
    def __str__(self):
        return f"- {self.status} {self.state_id:<3}" + ("\n"+9*" ").join([str(tr) for tr in self.transitions])
    

@dataclass
class Automata:
    start_state_id: str
    states: list[AutomataState]
    accepting_sink_sets_id: list[str]
    atomic_symbol_to_propositions: dict[str, int]
    
    def __str__(self):
        start = f">> {self.start_state_id}"
        states = "\n".join([str(st) for st in self.states])
        acc = ", ".join(self.accepting_sink_sets_id)
        f_set = "{" + acc + "}"
        translation = json.dumps(self.atomic_symbol_to_propositions, indent=4)
        return f"{start}\n{states}\n\nF: {f_set}\n{translation}"
    
    @classmethod
    def from_hoa(cls, hoa_header, hoa_states: list[HOAAutomataState]):
        start_state_id = hoa_header['start_state_id']
        accepting_sink_components = hoa_header['accepting_sink_sets_id']
        propositions_translation_dict = hoa_header['atomic_symbol_to_propositions']
        last_state_id = len(hoa_states) - 1
        org_states = []
        synth_states = []
        
        for state in hoa_states:
            trans = []
            for tr in state.transitions:
                if tr.accepting_signature:
                    last_state_id += 1
                    epsilon_tr = AutomataTransition(
                        type=AutomataTransitionType.Epsilon,
                        destination_id=tr.destination,
                        predicate=""
                    )
                    new_state = AutomataState(
                        state_id=last_state_id,
                        status=AutomataStateStat.Accepting,
                        transitions=[epsilon_tr],
                        accepting_signature=tr.accepting_signature
                    )
                    synth_states.append(new_state)
                    
                    new_tr = AutomataTransition(
                        type=AutomataTransitionType.Propositional,
                        destination_id=last_state_id,
                        predicate=tr.label
                    )
                else:
                    new_tr = AutomataTransition(
                        type=AutomataTransitionType.from_str(tr.label),
                        destination_id=tr.destination,
                        predicate=tr.label,
                    )
                trans.append(new_tr)
            new_st = AutomataState(
                state_id=state.state_id,
                status=AutomataStateStat.NonAccepting,
                transitions=trans,
                accepting_signature=[]
            )
            org_states.append(new_st)
        
        states = org_states + synth_states
        return cls(
            start_state_id=start_state_id,
            states=states,
            accepting_sink_sets_id=accepting_sink_components,
            atomic_symbol_to_propositions=propositions_translation_dict
        )
            
            

# Test them

In [43]:
parser = HOAParserHelper(hoa_parser)

In [44]:
# ldba = run_ltl2ldba("./rabinizer/owl", "F G a | G F b")
ldba = run_ltl2ldba("./rabinizer/owl", "G (a & (b U (c U (d | e))))")

print(ldba)

HOA: v1
tool: "owl ltl2ldgba" "21.0"
name: "Automaton for ((G(a)) & (G(((b) U (((c) U (((d) | (e)))))))))"
owlArgs: "ltl2ldgba" "-f" "G (a & (b U (c U (d | e))))" 
Start: 0
acc-name: generalized-Buchi 2
Acceptance: 2 Inf(0) & Inf(1)
properties: trans-acc no-univ-branch 
AP: 5 "a" "b" "c" "d" "e"
--BODY--
State: 0
[0 & !1 & 2 & !3 & !4] 1
[0 & (1 | !1 & (3 | !3 & 4))] 0
[0 & !1 & 2 & !3 & !4] 2
[0 & !1 & 2 & !3 & !4] 3
[0 & (1 | !1 & (3 | !3 & 4))] 4
[0 & (1 | !1 & (3 | !3 & 4))] 5
State: 1
[0 & 2 & !3 & !4] 1
[0 & !3 & 4] 6
[0 & 2 & !3 & !4] 7
[0 & 3 & !4] 8
[0 & 2 & !3 & !4] 9
[0 & (3 | !3 & 4)] 0
[0 & 4] 4
[0 & 3] 5
State: 2
[0 & 2 & !3 & !4] 2
[0 & 3] 5 {0 1}
[0 & !3 & 4] 5 {0}
State: 3
[0 & 4] 4 {0 1}
[0 & 3 & !4] 4 {0}
[0 & 2 & !3 & !4] 3
State: 4
[0 & 4] 4 {0 1}
[0 & (1 & !4 | !1 & 3 & !4)] 4 {0}
[0 & !1 & 2 & !3 & !4] 3
State: 5
[0 & 3] 5 {0 1}
[0 & !1 & 2 & !3 & !4] 2
[0 & (1 & !3 | !1 & !3 & 4)] 5 {0}
State: 6
[0 & (1 & !3 | !1 & !3 & 4)] 6
[0 & !1 & 2 & !3 & !4] 10
[0 & 3] 5


In [45]:
parsed_ldba = parser(ldba)

In [46]:
parsed_ldba['header']

{'start_state_id': '0',
 'accepting_sink_sets_id': ['0', '1'],
 'atomic_symbol_to_propositions': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}}

In [47]:
for st in parsed_ldba['states']:
    print(st)

(0)   -> 0&!1&2&!3&!4 -> (1)
      -> 0&(1|!1&(3|!3&4)) -> (0)
      -> 0&!1&2&!3&!4 -> (2)
      -> 0&!1&2&!3&!4 -> (3)
      -> 0&(1|!1&(3|!3&4)) -> (4)
      -> 0&(1|!1&(3|!3&4)) -> (5)
(1)   -> 0&2&!3&!4 -> (1)
      -> 0&!3&4 -> (6)
      -> 0&2&!3&!4 -> (7)
      -> 0&3&!4 -> (8)
      -> 0&2&!3&!4 -> (9)
      -> 0&(3|!3&4) -> (0)
      -> 0&4 -> (4)
      -> 0&3 -> (5)
(2)   -> 0&2&!3&!4 -> (2)
      -> 0&3 -> (5) ACC[0, 1]
      -> 0&!3&4 -> (5) ACC[0]
(3)   -> 0&4 -> (4) ACC[0, 1]
      -> 0&3&!4 -> (4) ACC[0]
      -> 0&2&!3&!4 -> (3)
(4)   -> 0&4 -> (4) ACC[0, 1]
      -> 0&(1&!4|!1&3&!4) -> (4) ACC[0]
      -> 0&!1&2&!3&!4 -> (3)
(5)   -> 0&3 -> (5) ACC[0, 1]
      -> 0&!1&2&!3&!4 -> (2)
      -> 0&(1&!3|!1&!3&4) -> (5) ACC[0]
(6)   -> 0&(1&!3|!1&!3&4) -> (6)
      -> 0&!1&2&!3&!4 -> (10)
      -> 0&3 -> (5)
(7)   -> 0&2&!3&!4 -> (7)
      -> 0&3&!4 -> (8)
      -> 0&4 -> (4)
(8)   -> 0&(1&!4|!1&3&!4) -> (8)
      -> 0&!1&2&!3&!4 -> (11)
      -> 0&4 -> (4)
(9)   -> 0&!3&4

In [48]:
automata = Automata.from_hoa(parsed_ldba['header'], parsed_ldba['states'])

print(automata)

>> 0
- ( ) 0  --[0 & !1 & 2 & !3 & !4]--> 1
         --[0 & (1 | !1 & (3 | !3 & 4))]--> 0
         --[0 & !1 & 2 & !3 & !4]--> 2
         --[0 & !1 & 2 & !3 & !4]--> 3
         --[0 & (1 | !1 & (3 | !3 & 4))]--> 4
         --[0 & (1 | !1 & (3 | !3 & 4))]--> 5
- ( ) 1  --[0 & 2 & !3 & !4]--> 1
         --[0 & !3 & 4]--> 6
         --[0 & 2 & !3 & !4]--> 7
         --[0 & 3 & !4]--> 8
         --[0 & 2 & !3 & !4]--> 9
         --[0 & (3 | !3 & 4)]--> 0
         --[0 & 4]--> 4
         --[0 & 3]--> 5
- ( ) 2  --[0 & 2 & !3 & !4]--> 2
         --[0 & 3]--> 12
         --[0 & !3 & 4]--> 13
- ( ) 3  --[0 & 4]--> 14
         --[0 & 3 & !4]--> 15
         --[0 & 2 & !3 & !4]--> 3
- ( ) 4  --[0 & 4]--> 16
         --[0 & (1 & !4 | !1 & 3 & !4)]--> 17
         --[0 & !1 & 2 & !3 & !4]--> 3
- ( ) 5  --[0 & 3]--> 18
         --[0 & !1 & 2 & !3 & !4]--> 2
         --[0 & (1 & !3 | !1 & !3 & 4)]--> 19
- ( ) 6  --[0 & (1 & !3 | !1 & !3 & 4)]--> 6
         --[0 & !1 & 2 & !3 & !4]--> 10
         --[0 